In [9]:
using LinearAlgebra

function conjgrad(forward::Function, adjoint::Function, shaping::Function, d::Array, p0::Array; ϵ=1.0, niter=1, tolerance= 1.0e-7)
"Conjugate-gradient algorithm for shaping regularization"
p = deepcopy(p0)
x = shaping(p)
r = forward(x) .- d      
sp, sx, sr = similar(p), similar(x), similar(r)
gnp, g0 = zero(Float64), zero(Float64)
for iter in 1:niter
gx = adjoint(r) - ϵ*x
gp = shaping(gx) + ϵ*p
gx = shaping(gp)
gr = forward(gx)

gn = gp ⋅ gp
@show iter, gn

if iter==1
g0 = gn

sp = gp
sx = gx
sr = gr
else
γ = gn/g0
β = gn/gnp

if γ < tolerance || β < tolerance
  # converged
  @show γ, β
  break
end

sp = gp + β*sp
sx = gx + β*sx
sr = gr + β*sr
end
gnp = gn

α = sr ⋅ sr + ϵ*(sp ⋅ sp -  sx ⋅ sx)
α = - gn/α

p = p + α*sp
x = x + α*sx
r = r + α*sr
end
return x
end

conjgrad (generic function with 1 method)

In [3]:
function doubint!(x::Vector{T}, der=false) where T <: Real
    "causal and anticausal integration in place"
    n = length(x)  
    ranges = der ? (1:n, ) : (1:n, n:-1:1)
    for range in ranges
        t = zero(T)
        for i in range
            t += x[i]
            x[i] = t
        end
    end
end

function fold(t::Vector{T}, nb::Integer) where T <: Real 
    "reflecting boundary conditions for smoothing"
    nt = length(t)
    n = nt - 2*nb
    # copy middle
    x = t[1+nb:n+nb]
    # reflections from the right side 
    for j in nb+n:2*n:nt
        for i in 1:min(n,nt-j); x[n+1-i] += t[j+i]; end
        for i in 1:min(n,nt-j-n); x[i] += t[j+n+i]; end
    end
    # reflections from the left side
    for j in nb+1:-2*n:1
        for i in 1:min(n,j-1); x[i] += t[j-i]; end
        for i in 1:min(n,j-1-n); x[n+1-i] += t[j-i-n]; end
    end
    return x
end

function smooth(x::Vector{T}, nb::Int, der=false) where T <: Real
    "smoothing by triangle filtering with reflecting boundaries"
    n = length(x)
    t = zeros(T,n+2*nb)
    for i in 1:n
        xi = x[i]/(nb*nb)
        t[i] -= xi
        t[i+nb] += 2*xi
        t[i+2*nb] -= xi
    end
    doubint!(t, der)
    return fold(t, nb)
end

smooth (generic function with 2 methods)

In [4]:
function smooth(x::Vector{T}, nb::Int, der=false) where T <: Real
    "smoothing by triangle filtering with reflecting boundaries"
    n = length(x)
    t = zeros(T,n+2*nb)
    for i in 1:n
        xi = x[i]/(nb*nb)
        t[i] -= xi
        t[i+nb] += 2*xi
        t[i+2*nb] -= xi
    end
    doubint!(t, der)
    return fold(t, nb)
end

smooth (generic function with 2 methods)

In [5]:
function smooth(x::Array, nb::Vector{Int}, der=false) 
    "multidimensional smoothing"
    y = deepcopy(x)
    # loop over dimensions
    for dim in 1:length(nb)
        y = mapslices(slice -> smooth(slice, nb[dim], der), y; dims=dim)
    end
    return y
end

smooth (generic function with 4 methods)

In [12]:
const DEFAULT_ϵ = 1.0
const DEFAULT_TOLERANCE = 1.0e-7
const DEFAULT_NITER = 100

function smooth_division(numerator::Array, denominator::Array, radius::Vector{Int}; 
                        niter=DEFAULT_NITER, ϵ=DEFAULT_ϵ, tolerance=DEFAULT_TOLERANCE)
    n = length(numerator)
    p0 = similar(numerator)
    fill!(p0, zero(eltype(p0)))
    # normalization
    norm = denominator ⋅ denominator
    if norm == 0.0
        return zeros(n, eltype(numerator))
    end
    norm = sqrt(n / norm)
    @show norm
    # weighting function
    weight = x -> x .* denominator .* norm
    return conjgrad(weight, weight, x -> smooth(x, radius), numerator * norm, p0; 
                    ϵ=ϵ, niter=niter, tolerance=tolerance)
end

smooth_division (generic function with 1 method)

In [13]:
# [TEST CASE 4]: Test for Smooth Division function
numerator = [1.0, 2.0, 3.0, 4.0]
denominator = [0.1, 0.2, 0.3, 0.4]
radius = [5]
output_smooth_division = smooth_division(numerator, denominator, radius)

norm = 3.651483716701107
(iter, gn) = (1, 412.0291555555554)
(iter, gn) = (2, 9.915087774665228)
(iter, gn) = (3, 0.00032678726342515705)
(iter, gn) = (4, 5.309053069195088e-12)
(γ, β) = (1.2885139310194395e-14, 1.624620560039361e-8)


4-element Vector{Float64}:
  9.99999999483288
 10.00000001181633
  9.999999990811466
 10.000000002537154